In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
def get_p_a(df, res, tot):
    p_a_list = []
    for index, row in df.iterrows():
        p_ai = 0
        for column in df.columns:
            value = df.loc[index][column]
            p_aik = (value*(value-1)) / (res*(res-1))
            p_ai = p_ai + p_aik
        p_a_list.append(p_ai)
        
    p_as = sum(p_a_list) / len(p_a_list)
    p_a = p_as * (1 - (1/tot)) + (1/tot)
    
    return p_a

In [3]:
def get_p_e(df, tot):
    p_es_list = []
    for column in df.columns:
        p_ei = df[column].sum() / tot
        p_es_list.append(p_ei)
    
    p_e_list = [p_e ** 2 for p_e in p_es_list]
    p_e = sum(p_e_list)
    
    return p_e

In [4]:
def krippendorff_alpha(df, res, tot, p_e_in):
    p_a = get_p_a(df, res, tot)
    #p_e = get_p_e(df, tot)
    p_e = p_e_in
    
    alpha = (p_a - p_e) / (1 - p_e)
    
    return alpha

In [5]:
def create_agreement_matrix(df, ratings):
    df_k = pd.DataFrame(columns = ratings)
    for column in df.columns:
        values_list = []
        for rating in ratings:
            value = (df[column]==rating).sum()
            values_list.append(value)
        df_k_len = len(df_k)
        df_k.loc[df_k_len] = values_list
    rows = df.columns
    df_k['Question'] = rows
    
    return df_k

In [6]:
def data_preprocessing(df, criteria):
    df = df.drop(['ID','Begintijd', 'Tijd van voltooien', 'E-mail', 'Naam'], axis=1)
    df = df.drop(['What is your name? (this will be anonymized when the data of all four parts is combined)'], axis=1)
    
    column_list = []
    for column in df.columns:
        if criteria in column:
            column_id = df.columns.get_loc(column)
            column_list.append(column_id)
    
    new_df = df.iloc[: , column_list].copy()
        
    return new_df

In [7]:
def combine_frames(frames, criteria, ratings):
    frames_k = []
    
    for frame in frames:
        clean_frame = data_preprocessing(frame, criteria)
        frame_k = create_agreement_matrix(clean_frame, ratings)
        frames_k.append(frame_k)
    
    final_k = pd.concat(frames_k).reset_index()
    final_f = final_k.drop(['index', 'Question'], axis=1)
    
    return final_f

In [8]:
general = pd.read_excel('Vattenfall/Material/Requirements validation & mapping part 1_ General (1-9).xlsx')
ingestion = pd.read_excel('Vattenfall/Material/Requirements validation & mapping part 2_ Ingestion(1-9).xlsx')
calculation = pd.read_excel('Vattenfall/Material/Requirements validation & mapping part 3_ Calculation(1-9).xlsx')
export = pd.read_excel('Vattenfall/Material/Requirements validation & mapping part 4_ Export(1-9).xlsx')

In [9]:
frames = [general, ingestion, calculation, export]

## Understandability quality measure

In [10]:
ratings = ['Yes', 'No']
criteria = 'understand'

In [11]:
final_f_u = combine_frames(frames, criteria, ratings)

In [12]:
len(final_f_u)

56

In [13]:
final_f_u.to_csv('Vattenfall/Material/understandability_data.csv')

In [14]:
res = 9
tot = 504
p_e_in = 0.0
alpha = krippendorff_alpha(final_f_u, res, tot, p_e_in)
print("Krippendorff's alpha: ", alpha)

Krippendorff's alpha:  0.9445546737213406


In [15]:
total_y = final_f_u['Yes'].sum()
total_n = final_f_u['No'].sum()
print(total_y)
print(total_n)

488
16


In [16]:
count = 0
for index, row in final_f_u.iterrows():
    if final_f_u.iloc[index, 0] >= 4.5:
        count += 1
print(count)

56


In [17]:
count = 0
for index, row in final_f_u.iterrows():
    if final_f_u.iloc[index, 0] == 9:
        count += 1
print(count)

45


In [18]:
for index, row in final_f_u.iterrows():
    if final_f_u.iloc[index, 0] <= final_f_u.iloc[index, 1]:
        print("FR"+str(index+1))

## Correctness quality measure

In [19]:
ratings = ['Yes', 'No']
criteria = 'satisfy'

In [20]:
final_f_c = combine_frames(frames, criteria, ratings)

In [21]:
len(final_f_c)

56

In [22]:
final_f_c.to_csv('Vattenfall/Material/correctness_data.csv')

In [23]:
total = final_f_c.sum(axis=1)
average = total.mean()
print(average)

8.678571428571429


In [24]:
res = 8.7
tot = 487.2
p_e_in = 0.0
alpha = krippendorff_alpha(final_f_c, res, tot, p_e_in)
print("Krippendorff's alpha: ", alpha)

Krippendorff's alpha:  0.9033187194814508


In [25]:
total_y = final_f_c['Yes'].sum()
total_n = final_f_c['No'].sum()
print(total_y)
print(total_n)

461
25


In [26]:
print(final_f_c.iloc[0].sum())

8


In [27]:
count = 0
for index, row in final_f_c.iterrows():
    if final_f_c.iloc[index, 0] == final_f_c.iloc[index].sum():
        count += 1
print(count)

37


In [28]:
count = 0
for index, row in final_f_c.iterrows():
    if final_f_c.iloc[index, 0] >= (final_f_c.iloc[index].sum()/2):
        count += 1
print(count)

56


## Completeness quality measure

In [29]:
def completeness_conversion(df):
    df = df.fillna('')
    df['state'] = ''
    for index, row in df.iterrows():
        if len(df.iloc[index, 0]) == 0 or df.iloc[index, 0] == 'No':
            df.iloc[index, 1] = 'No'
        else:
            df.iloc[index, 1] = 'Yes'
            
    return df

In [30]:
def combine_frames_completeness(frames, criteria, ratings):
    k_frames = []
    for frame in frames:
        new_frame = data_preprocessing(frame, criteria)
        converted_frame = completeness_conversion(new_frame)
        
        column = converted_frame.columns[0]
        converted_frame = converted_frame.drop(column, axis=1)
        k_frame = create_agreement_matrix(converted_frame, ratings)
        
        k_frames.append(k_frame)
    
    completeness_k = pd.concat(k_frames).reset_index()
    completeness_f = completeness_k.drop(['index', 'Question'], axis=1)
    
    return completeness_f

In [31]:
ratings = ['No', 'Yes']
criteria = 'missed'

In [32]:
completeness_f = combine_frames_completeness(frames, criteria, ratings)

In [33]:
completeness_f

,No,Yes
0,5,4
1,7,2
2,7,2
3,7,2


In [34]:
completeness_f.to_csv('Vattenfall/Material/completeness_data.csv')

In [35]:
res = 9
tot = 36
p_e_in = 0.0
alpha = krippendorff_alpha(completeness_f, res, tot, p_e_in)
print("Krippendorff's alpha: ", alpha)

Krippendorff's alpha:  0.5814043209876543


In [36]:
total_y = completeness_f['Yes'].sum()
total_n = completeness_f['No'].sum()
print(total_y)
print(total_n)

10
26


In [37]:
count = 0
for index, row in completeness_f.iterrows():
    if completeness_f.iloc[index, 0] == 9:
        count += 1
print(count)

0


In [38]:
count = 0
for index, row in completeness_f.iterrows():
    if completeness_f.iloc[index, 0] >= 4.5:
        count += 1
print(count)

4


## Plot

In [39]:
index_list = final_f_u.index.tolist() 
x = []
for element in index_list:
    FR = 'FR'+str(element + 1)  
    x.append(FR)

In [40]:
f = plt.figure()
f.set_figwidth(40)
f.set_figheight(10)
f.set_dpi(500)
plt.plot(x, final_f_u['Yes'].tolist(), label = 'Understandability')
plt.plot(x, final_f_c['Yes'].tolist(), label = 'Correctness')
plt.xticks(fontsize=20, rotation=90)
plt.yticks(fontsize=20)
plt.xlabel('Functional requirement', fontsize=20)
plt.ylabel('Number of \'Yes\' ratings', fontsize=20)
plt.legend(fontsize=20)
plt.savefig('quality_over_requirements.png')
plt.show()

## Functionality mapping

In [41]:
ratings = ['Yes', 'No', 'Partially']
criteria = 'current'

In [42]:
final_f_fm = combine_frames(frames, criteria, ratings)

In [43]:
len(final_f_fm)

56

In [44]:
final_f_fm.to_csv('Vattenfall/Material/func_mapping_data.csv')

In [45]:
res = 9
tot = 504
p_e_in = 0.0
alpha = krippendorff_alpha(final_f_fm, res, tot, p_e_in)
print("Krippendorff's alpha: ", alpha)

Krippendorff's alpha:  0.4509922603300578


In [46]:
total_y = final_f_fm['Yes'].sum()
total_n = final_f_fm['No'].sum()
total_p = final_f_fm['Partially'].sum()
print(total_y)
print(total_n)
print(total_p)

231
172
101


In [47]:
count = 0
for index, row in final_f_fm.iterrows():
    if final_f_fm.iloc[index, 0] == 9:
        count += 1
print(count)

0


In [48]:
count = 0
for index, row in final_f_fm.iterrows():
    if final_f_fm.iloc[index, 0] >= 4.5:
        count += 1
print(count)

28
